#### Import the necessary Libraries

In [13]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#### File open the CSV files and bring them in as their own Data Frames

In [14]:
members =pd.read_csv('../data/NSS_GreatCare_Members.csv')


In [15]:
admissions =pd.read_csv('../data/NSS_Hospital_Inpatient_Admissions.csv')

In [16]:
pcp_visit =pd.read_csv('../data/NSS_PrimaryCare_Office_Visits.csv')

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [17]:
print('\nMembers Data\n\n')
print(members.info())
print('\n\n------------------------------------------------------------------------------------------\n\n')
print('Admissions Data\n\n')
print(admissions.info())
print('\n\n------------------------------------------------------------------------------------------\n\n')
print('PCP Visit\n\n')
print(pcp_visit.info())


Members Data


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61519 entries, 0 to 61518
Data columns (total 63 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   Patient ID                                                       61519 non-null  object 
 1   Primary Plan ID                                                  61519 non-null  object 
 2   Primary Payor ID                                                 61519 non-null  object 
 3   Is Patient Alive                                                 61519 non-null  int64  
 4   Patient Age Integer                                              61519 non-null  int64  
 5   Patient Date of Birth                                            61519 non-null  object 
 6   Patient ZIP Code 5-Digit                                         61105 non-null  float64
 7   Count Primary Care Visit

#### Clean up data fields - Getting the House in Orde

In [18]:
# Change Object data type to Dates
members['Period Start Date'] = pd.to_datetime(members['Period Start Date'], format='%Y-%m-%d')
members['Period End Date'] = pd.to_datetime(members['Period End Date'], format='%Y-%m-%d')
members['Patient Date of Birth'] = pd.to_datetime(members['Patient Date of Birth'], format='%Y-%m-%d')
# Drop the Zeros on the Zip and fill in the blank and convert to the string
members['Patient ZIP Code 5-Digit'] = members['Patient ZIP Code 5-Digit'].fillna("0").astype(int).astype(str)
# Add an additional field of 30days after the end date.
#members['days_30'] = members['Period End Date']+ pd.DateOffset(days=30)
# Change Object data type to Dates
admissions['Encounter Start Date'] = pd.to_datetime(admissions['Encounter Start Date'], format='%Y-%m-%d')
admissions['Encounter End Date'] = pd.to_datetime(admissions['Encounter End Date'], format='%Y-%m-%d')
pcp_visit['Encounter Date'] = pd.to_datetime(pcp_visit['Encounter Date'], format='%Y-%m-%d')

### Take select columns from the dataframes

In [19]:
admissions1 = admissions[['Encounter ID', 'Patient ID', 'Attending Provider NPI',
       'Facility Provider NPI', 'Encounter Start Date', 'Encounter End Date',
       #'Principal Discharge Diagnosis ICD-9-CM Code',
       #'Principal Discharge Diagnosis ICD-9-CM Description',
       #'Principal Discharge Diagnosis ICD-10-CM Code',
       #'Principal Discharge Diagnosis ICD-10-CM Description',
       #'Primary Plan Allowed Amount', 'Primary Plan Paid Amount',
       'Patient Paid Amount']]

In [20]:
admissions1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6213 entries, 0 to 6212
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Encounter ID            6213 non-null   object        
 1   Patient ID              6213 non-null   object        
 2   Attending Provider NPI  5884 non-null   float64       
 3   Facility Provider NPI   6213 non-null   object        
 4   Encounter Start Date    6213 non-null   datetime64[ns]
 5   Encounter End Date      6213 non-null   datetime64[ns]
 6   Patient Paid Amount     6213 non-null   float64       
dtypes: datetime64[ns](2), float64(2), object(3)
memory usage: 339.9+ KB


In [21]:
#reduce pcp to relevant columns
pcp1 = pcp_visit[['Encounter ID','Patient ID', 'Attending Provider NPI', 'Service Provider NPI',
          'Provider Group NPI','Encounter Date']]


In [22]:
pcp1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231373 entries, 0 to 231372
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Encounter ID            231373 non-null  object        
 1   Patient ID              231373 non-null  object        
 2   Attending Provider NPI  39644 non-null   float64       
 3   Service Provider NPI    230599 non-null  float64       
 4   Provider Group NPI      227644 non-null  float64       
 5   Encounter Date          231373 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 10.6+ MB


### Problem 1: 30-day Readmission Rate

#### What is our 30-day all cause readmission rate across hospitals our members are admitted to?¶

#### Getting the next admission date for same patient to same line

In [23]:
admissions2 = admissions1
admissions2['next_adm']= admissions2.sort_values(by=['Encounter Start Date'],ascending=True).groupby(['Patient ID']) ['Encounter Start Date'].shift(-1)
admissions2.head(50)
# admissions2['next_adm']= admissions2.sort_values(by=['Encounter Start Date'],ascending=True).groupby(['Patient ID']) ['Encounter Start Date'].shift(-1)
# admissions2.tail(50)

<ipython-input-23-0380320bb9d9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  admissions2['next_adm']= admissions2.sort_values(by=['Encounter Start Date'],ascending=True).groupby(['Patient ID']) ['Encounter Start Date'].shift(-1)


,Encounter ID,Patient ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,Patient Paid Amount,next_adm
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,1.417171e+09,1306890389,2013-01-12,2013-01-14,250.0,NaT
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,1.669599e+09,1902803315,2014-01-05,2014-01-12,250.0,NaT
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,1.407881e+09,1902803315,2012-06-13,2012-06-21,250.0,NaT
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,1.407009e+09,1427055839,2012-12-06,2012-12-09,250.0,NaT
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,1.518046e+09,1992818256,2015-03-11,2015-03-14,0.0,NaT
5,HIA|DS1|10126|DS1|NPI|1902803315|1,DS1|10126,1.346557e+09,1902803315,2010-04-07,2010-04-09,250.0,NaT
6,HIA|DS1|10130|DS1|NPI|1427055839|1,DS1|10130,1.316989e+09,1427055839,2014-06-01,2014-06-05,250.0,NaT
7,HIA|DS1|10132|DS1|NPI|1366545311|1,DS1|10132,NaN,1366545311,2013-08-14,2013-08-16,250.0,NaT
8,HIA|DS1|10132|DS1|NPI|1386741635|1,DS1|10132,1.417995e+09,1386741635,2013-03-30,2013-04-01,250.0,2013-08-14
9,HIA|DS1|10135|DS1|NPI|1184722779|1,DS1|10135,1.396818e+09,1184722779,2013-08-20,2013-08-22,250.0,NaT


In [24]:
#testing output
admissions2[admissions2['Patient ID']=='DS1|10451']

,Encounter ID,Patient ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,Patient Paid Amount,next_adm
34,HIA|DS1|10451|DS1|NPI|1023010113|1,DS1|10451,1.306903e+09,1023010113,2014-04-24,2014-04-26,250.0,2014-05-23
35,HIA|DS1|10451|DS1|NPI|1063412005|1,DS1|10451,1.518963e+09,1063412005,2013-03-20,2013-03-23,250.0,2013-03-27
36,HIA|DS1|10451|DS1|NPI|1225199987|1,DS1|10451,1.750425e+09,1225199987,2014-06-29,2014-06-29,0.0,NaT
37,HIA|DS1|10451|DS1|NPI|1427055839|1,DS1|10451,1.427345e+09,1427055839,2013-04-16,2013-04-26,50.0,2013-05-01
38,HIA|DS1|10451|DS1|NPI|1427055839|2,DS1|10451,1.073567e+09,1427055839,2013-05-01,2013-05-09,0.0,2013-06-25
39,HIA|DS1|10451|DS1|NPI|1427055839|3,DS1|10451,1.073567e+09,1427055839,2013-06-25,2013-07-01,0.0,2013-07-26
40,HIA|DS1|10451|DS1|NPI|1427055839|4,DS1|10451,1.679542e+09,1427055839,2013-07-26,2013-08-03,0.0,2013-08-31
41,HIA|DS1|10451|DS1|NPI|1427055839|5,DS1|10451,1.215190e+09,1427055839,2013-08-31,2013-09-02,0.0,2013-11-14
42,HIA|DS1|10451|DS1|NPI|1427055839|6,DS1|10451,1.679542e+09,1427055839,2013-11-14,2013-11-20,0.0,2013-12-17
43,HIA|DS1|10451|DS1|NPI|1427055839|7,DS1|10451,1.053358e+09,1427055839,2013-12-17,2013-12-19,250.0,2014-04-24


#### Now that we have the Next Encounter Start Date added to the Current Encounter row. We can Calculate the number of days between the two visits

In [25]:
admissions2['Days Between Readmission']= admissions2['next_adm'] - admissions2['Encounter End Date']
admissions2

<ipython-input-25-5d65673b1a92>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  admissions2['Days Between Readmission']= admissions2['next_adm'] - admissions2['Encounter End Date']


,Encounter ID,Patient ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,Patient Paid Amount,next_adm,Days Between Readmission
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,1.417171e+09,1306890389,2013-01-12,2013-01-14,250.0,NaT,NaT
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,1.669599e+09,1902803315,2014-01-05,2014-01-12,250.0,NaT,NaT
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,1.407881e+09,1902803315,2012-06-13,2012-06-21,250.0,NaT,NaT
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,1.407009e+09,1427055839,2012-12-06,2012-12-09,250.0,NaT,NaT
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,1.518046e+09,1992818256,2015-03-11,2015-03-14,0.0,NaT,NaT
...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,1.720043e+09,1689772592,2015-02-06,2015-02-11,250.0,NaT,NaT
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,1.215968e+09,1427055839,2014-08-07,2014-08-10,250.0,NaT,NaT
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,1.861832e+09,1689608150,2015-11-21,2015-11-23,250.0,2015-11-24,1 days
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,1.750545e+09,1689608150,2015-11-24,2015-11-27,250.0,NaT,NaT


#### Drop all rows that have no Values in the Admission 2 Data Frame in the 'next_adm' or 'Days between Readmission' as they were not Re-admitted

In [26]:
re_admits = admissions2.dropna(subset=['next_adm'])
re_admits

,Encounter ID,Patient ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,Patient Paid Amount,next_adm,Days Between Readmission
8,HIA|DS1|10132|DS1|NPI|1386741635|1,DS1|10132,1.417995e+09,1386741635,2013-03-30,2013-04-01,250.0,2013-08-14,135 days
18,HIA|DS1|10253|DS1|NPI|1396725677|1,DS1|10253,1.932127e+09,1396725677,2010-06-02,2010-06-06,250.0,2011-10-15,496 days
19,HIA|DS1|10253|DS1|NPI|1689772592|1,DS1|10253,1.346566e+09,1689772592,2010-04-02,2010-04-16,250.0,2010-06-02,47 days
20,HIA|DS1|10291|DS1|NPI|1821250762|1,DS1|10291,1.598994e+09,1821250762,2010-05-14,2010-05-15,250.0,2011-02-24,285 days
34,HIA|DS1|10451|DS1|NPI|1023010113|1,DS1|10451,1.306903e+09,1023010113,2014-04-24,2014-04-26,250.0,2014-05-23,27 days
...,...,...,...,...,...,...,...,...,...
6198,HIA|DS1|9935|DS1|NPI|1669456299|1,DS1|9935,1.376864e+09,1669456299,2011-01-22,2011-01-24,250.0,2011-01-25,1 days
6199,HIA|DS1|9935|DS1|NPI|1902803315|1,DS1|9935,1.972545e+09,1902803315,2011-01-25,2011-01-26,250.0,2011-01-26,0 days
6201,HIA|DS1|9943|DS1|NPI|1184722779|1,DS1|9943,1.932297e+09,1184722779,2013-10-01,2013-10-16,160.0,2013-10-25,9 days
6202,HIA|DS1|9943|DS1|NPI|1184722779|2,DS1|9943,1.942320e+09,1184722779,2013-10-25,2013-10-28,0.0,2014-10-28,365 days


In [27]:
# the Dataframe does contain negative values in the Days between Readmission as well as we need to 
# find the records that are 0 - 30 days.
re_admits = re_admits[(re_admits['Days Between Readmission'] <= dt.timedelta(days = 30)) & (re_admits['Days Between Readmission'] >= dt.timedelta(days = 0)) ]
re_admits.head(50) 

,Encounter ID,Patient ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,Patient Paid Amount,next_adm,Days Between Readmission
34,HIA|DS1|10451|DS1|NPI|1023010113|1,DS1|10451,1.306903e+09,1023010113,2014-04-24,2014-04-26,250.0,2014-05-23,27 days
35,HIA|DS1|10451|DS1|NPI|1063412005|1,DS1|10451,1.518963e+09,1063412005,2013-03-20,2013-03-23,250.0,2013-03-27,4 days
37,HIA|DS1|10451|DS1|NPI|1427055839|1,DS1|10451,1.427345e+09,1427055839,2013-04-16,2013-04-26,50.0,2013-05-01,5 days
39,HIA|DS1|10451|DS1|NPI|1427055839|3,DS1|10451,1.073567e+09,1427055839,2013-06-25,2013-07-01,0.0,2013-07-26,25 days
40,HIA|DS1|10451|DS1|NPI|1427055839|4,DS1|10451,1.679542e+09,1427055839,2013-07-26,2013-08-03,0.0,2013-08-31,28 days
42,HIA|DS1|10451|DS1|NPI|1427055839|6,DS1|10451,1.679542e+09,1427055839,2013-11-14,2013-11-20,0.0,2013-12-17,27 days
45,HIA|DS1|10451|DS1|NPI|1902844988|1,DS1|10451,1.811123e+09,1902844988,2013-03-27,2013-04-16,250.0,2013-04-16,0 days
51,HIA|DS1|10507|DS1|NPI|1902803315|1,DS1|10507,1.750312e+09,1902803315,2012-11-10,2012-11-16,250.0,2012-11-20,4 days
66,HIA|DS1|10698|DS1|NPI|1902803315|1,DS1|10698,1.295777e+09,1902803315,2012-01-03,2012-01-04,250.0,2012-01-04,0 days
70,HIA|DS1|10707|DS1|NPI|1033298617|2,DS1|10707,1.932231e+09,1033298617,2010-07-25,2010-07-27,0.0,2010-08-13,17 days


In [29]:
# Identify the records in the Re-admission DF
repeat_admissons = len(re_admits.index)
print('Repeat Admissions count is ',repeat_admissons)
# Identify the records in the admissions DF
total_hospitalizations = len(admissions1.index)
print('Total Admissions count is ',total_hospitalizations)
# To calculate the percentage
perc_readmission = round((repeat_admissons/total_hospitalizations) *100 , 3)
print('The percentage of Readmission in the last 30 days is ',perc_readmission)

Repeat Admissions count is  696
Total Admissions count is  6213
The percentage of Readmission in the last 30 days is  11.202


#### Problem 2. What percent of patients have a PCP visit within a week of discharge?

In [ ]:
# We will merge Re-admits data frame with pcp visit dataframe to determine the whether a patient visited their PCP.






#### 3. Do either vary by individual or group PCP's?

Problem 3 is pretty vaguely worded. I understand it as asking two questions:    
(a) Does the 30-day readmission rate vary by hospital (which is how I interpreted the word "individual" there)?    
(b) Does the 7-day PCP visit rate vary by PCP Provider Group?     
Is this how everyone else is approaching this problem?    

##### A. Lets us review re-hospitalization by Hospital group

In [31]:
re_admits_facility = re_admits.groupby(by="Facility Provider NPI",as_index=False)['Encounter ID'].count()
re_admits_facility

,Facility Provider NPI,Encounter ID
0,0000000000,14
1,1013062769,1
2,1013906221,2
3,1023010113,1
4,1023065729,2
...,...,...
84,1952546616,7
85,1962568337,1
86,1972541498,1
87,1992818256,10


In [32]:
# We will use the admissions1 Dataframe (ALL PATIENTS THAT WENT TO THE HOSPITAL) to group it by Hospital NPI to see if we create a denominator for the above Re-admissions data
Facility_admits = admissions1.groupby(by='Facility Provider NPI',as_index=False)['Encounter ID'].count()
Facility_admits


,Facility Provider NPI,Encounter ID
0,0000000000,103
1,1003878539,1
2,1003961251,2
3,1003980350,1
4,1013062769,1
...,...,...
215,EI26275047,2
216,EI26439240,1
217,EI95472322,1
218,EI95488125,3


In [38]:
#We will now merge the facility admits ( to have denominator) to the re-admits-facility ( numerator)to calculate
#rate of the % re-admission by facility NPI over time.

rehospitalization = pd.merge(Facility_admits, re_admits_facility, how ='left',on='Facility Provider NPI',suffixes =('_hosp', '_readm'))
#Select only the Ecounter ID columns
rehospitalization = rehospitalization[['Facility Provider NPI','Encounter ID_readm','Encounter ID_hosp']]

#Calulate the RE-Hospitalization rate 
rehospitalization['Re_hospitalization_rate'] = round((rehospitalization['Encounter ID_readm'] / rehospitalization['Encounter ID_hosp'] * 100 ) ,2 )
# Sort the Re-hospitalization rate
rehospitalization = rehospitalization.sort_values(['Re_hospitalization_rate'], ascending=False)
rehospitalization


,Facility Provider NPI,Encounter ID_readm,Encounter ID_hosp,Re_hospitalization_rate
66,1275661829,1.0,1,100.0
20,1073665360,1.0,1,100.0
45,1184612764,1.0,1,100.0
80,1376513754,2.0,2,100.0
162,1801152566,1.0,1,100.0
...,...,...,...,...
215,EI26275047,NaN,2,NaN
216,EI26439240,NaN,1,NaN
217,EI95472322,NaN,1,NaN
218,EI95488125,NaN,3,NaN


In [39]:
# to identify Hospital that have > 10 encounters but also have a high admission rate. 

rehospitalization = rehospitalization[rehospitalization['Encounter ID_hosp'] >= 10].sort_values(['Re_hospitalization_rate'], ascending=False)
rehospitalization


,Facility Provider NPI,Encounter ID_readm,Encounter ID_hosp,Re_hospitalization_rate
199,1952546616,7.0,10,70.00
62,1265814065,4.0,11,36.36
74,1336328244,3.0,10,30.00
177,1841277704,3.0,11,27.27
156,1770532608,3.0,11,27.27
131,1639172372,4.0,20,20.00
97,1467459776,3.0,16,18.75
165,1811080526,4.0,22,18.18
16,1053351916,2.0,11,18.18
153,1740348929,2.0,12,16.67


In [42]:
# check my work above
re_admits[re_admits['Facility Provider NPI'] == '1952546616']


,Encounter ID,Patient ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,Patient Paid Amount,next_adm,Days Between Readmission
438,HIA|DS1|13593|DS1|NPI|1952546616|1,DS1|13593,1.386652e+09,1952546616,2011-12-19,2011-12-22,250.00,2012-01-03,12 days
439,HIA|DS1|13593|DS1|NPI|1952546616|2,DS1|13593,1.386652e+09,1952546616,2012-01-03,2012-01-09,0.00,2012-01-26,17 days
440,HIA|DS1|13593|DS1|NPI|1952546616|3,DS1|13593,1.386652e+09,1952546616,2012-01-26,2012-02-09,250.00,2012-02-12,3 days
441,HIA|DS1|13593|DS1|NPI|1952546616|4,DS1|13593,1.386652e+09,1952546616,2012-02-12,2012-02-14,0.00,2012-02-22,8 days
442,HIA|DS1|13593|DS1|NPI|1952546616|5,DS1|13593,1.386652e+09,1952546616,2012-02-22,2012-03-11,209.61,2012-03-18,7 days
443,HIA|DS1|13593|DS1|NPI|1952546616|6,DS1|13593,1.386652e+09,1952546616,2012-03-18,2012-03-24,0.00,2012-04-17,24 days
444,HIA|DS1|13593|DS1|NPI|1952546616|7,DS1|13593,1.386652e+09,1952546616,2012-04-17,2012-05-06,0.00,2012-05-08,2 days
